In [1]:
import pickle
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import joblib
import os

In [ ]:
# load lunges data
with open("../data/data_3D.pickle", "rb") as f:
    data = pickle.load(f)

    labels = data["labels"]
    poses = data["poses"]

action_col = 0  # 'SQUAT' / 'Lunges' / 'Plank'
subject_col = 1  # subject name
instruction_col = 2 # instruction ID string
seq_col = 3  # sequence index

# filter to Lunges
mask_lunge = labels[:, action_col] == "Lunges"
labels_lunge = labels[mask_lunge]
poses_lunge = poses[mask_lunge]

print("Total frames in dataset:", labels.shape[0])
print("Lunge frames:", labels_lunge.shape[0])

codes_lunge = np.unique(labels_lunge[:, instruction_col])
print("Unique Lunge instruction codes:", codes_lunge)

code_seq_counts_lunge = {}
for c in codes_lunge:
    mask_c = labels_lunge[:, instruction_col] == c
    subj = labels_lunge[mask_c, subject_col]
    seq = labels_lunge[mask_c, seq_col]
    subj_seq = np.unique([f"{s}_{q}" for s, q in zip(subj, seq)])
    code_seq_counts_lunge[c] = len(subj_seq)
    print(f"code {c!r}: {len(subj_seq)} sequences")

Total frames in dataset: 29789
Lunge frames: 12754
Unique Lunge instruction codes: ['1' '4' '6']
code '1': 46 sequences
code '4': 40 sequences
code '6': 41 sequences


In [ ]:
instr_map_lunge = {
    "1": "LUNGE_CORRECT",
    "4": "LUNGE_NOT_LOW_ENOUGH",
    "6": "LUNGE_KNEE_PASSES_TOE",
}

In [4]:
# 3D features for lunge frames
N_lg, C, J = poses_lunge.shape
print("Lunge poses shape: N_lg, C, J =", N_lg, C, J)

root_idx = 0
root_lg = poses_lunge[:, :, root_idx][:, :, None] 
poses_lg_centered = poses_lunge - root_lg 

# flatten
X_lunge = poses_lg_centered.reshape(N_lg, -1)
print("X_lunge shape:", X_lunge.shape)

Lunge poses shape: N_lg, C, J = 12754 3 25
X_lunge shape: (12754, 75)


In [6]:
# encode instruction codes as classes
instr_codes_lg = labels_lunge[:, instruction_col]
print("Unique Lunge codes:", np.unique(instr_codes_lg))

# map each unique code to an integer
codes_lg = sorted(list(set(instr_codes_lg)))
code_to_id_lg = {c: i for i, c in enumerate(codes_lg)}
id_to_code_lg = {i: c for c, i in code_to_id_lg.items()}
print("code_to_id_lg:", code_to_id_lg)

y_lunge = np.array([code_to_id_lg[c] for c in instr_codes_lg], dtype=int)

Unique Lunge codes: ['1' '4' '6']
code_to_id_lg: {'1': 0, '4': 1, '6': 2}


In [7]:
# train/val/test split
X_train_lg, X_temp_lg, y_train_lg, y_temp_lg = train_test_split(
    X_lunge, y_lunge,
    test_size=0.3,
    stratify=y_lunge,
    random_state=42,
)

X_val_lg, X_test_lg, y_val_lg, y_test_lg = train_test_split(
    X_temp_lg, y_temp_lg,
    test_size=0.5,
    stratify=y_temp_lg,
    random_state=42,
)

print("Lunge train:", X_train_lg.shape[0])
print("Lunge val:", X_val_lg.shape[0])
print("Lunge test:", X_test_lg.shape[0])

Lunge train: 8927
Lunge val: 1913
Lunge test: 1914


In [8]:
# MLP
clf_lunge = make_pipeline(
    StandardScaler(),
    MLPClassifier(
        hidden_layer_sizes=(128, 64),
        activation="relu",
        alpha=1e-4,
        max_iter=400,
        early_stopping=True,
        n_iter_no_change=20,
        random_state=42,
    ),
)

clf_lunge.fit(X_train_lg, y_train_lg)

,steps,"[('standardscaler', ...), ('mlpclassifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,hidden_layer_sizes,"(128, ...)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001


In [9]:
# evaluation
target_names_lg = [f"instr_{id_to_code_lg[i]}" for i in sorted(id_to_code_lg.keys())]

print("lunge: valudation performance")
y_val_pred_lg = clf_lunge.predict(X_val_lg)
print(classification_report(y_val_lg, y_val_pred_lg, target_names=target_names_lg))

print("lunge: test performance")
y_test_pred_lg = clf_lunge.predict(X_test_lg)
print(classification_report(y_test_lg, y_test_pred_lg, target_names=target_names_lg))

lunge: valudation performance
              precision    recall  f1-score   support

     instr_1       1.00      1.00      1.00       751
     instr_4       0.99      0.98      0.99       551
     instr_6       0.99      1.00      0.99       611

    accuracy                           0.99      1913
   macro avg       0.99      0.99      0.99      1913
weighted avg       0.99      0.99      0.99      1913

lunge: test performance
              precision    recall  f1-score   support

     instr_1       1.00      0.99      1.00       752
     instr_4       0.99      0.99      0.99       551
     instr_6       0.99      0.99      0.99       611

    accuracy                           0.99      1914
   macro avg       0.99      0.99      0.99      1914
weighted avg       0.99      0.99      0.99      1914



In [12]:
# save model
os.makedirs("../models/lunge", exist_ok=True)

joblib.dump(clf_lunge, "../models/lunge/ec3d_lunge_instruction_mlp.pkl")
joblib.dump(id_to_code_lg, "../models/lunge/ec3d_lunge_instruction_code_map.pkl")

print("saved Lunge instruction model + code map.")

saved Lunge instruction model + code map.
